# Speech Emotion Recognition – Preprocessing
## Notebook : Group Pipeline

**Group Number:** 196

### Step 01 : Feature Extraction

In [ ]:
import os
import torch
import torchaudio
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

np.random.seed(42)

emotion_map = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mfcc_transform = torchaudio.transforms.MFCC(
    sample_rate=48000,
    n_mfcc=40,
    melkwargs={"n_fft": 2048, "hop_length": 512, "n_mels": 128}  # bigger FFT for higher quality
).to(device)

mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=48000,
    n_fft=2048,
    hop_length=512,
    n_mels=128
).to(device)

def extract_features(file_path):
    waveform, sr = torchaudio.load(file_path)

    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    if sr != 48000:
        waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=48000)(waveform)
        sr = 48000

    waveform = waveform.to(device).squeeze(0)

    mfcc = mfcc_transform(waveform.unsqueeze(0)).squeeze()

    mel = mel_transform(waveform.unsqueeze(0)).squeeze()

    waveform_np = waveform.cpu().numpy()
    chroma = librosa.feature.chroma_stft(
        y=waveform_np,
        sr=sr,
        n_fft=2048,
        hop_length=512
    )
    chroma = torch.tensor(chroma, device=device, dtype=torch.float32)

    return mfcc, mel, chroma

ravdess_dir = "../data/ravdess"
mfcc_features, mel_features, chroma_features, emotion_labels = [], [], [], []

for root, dirs, files in os.walk(ravdess_dir):
    for file in files:
        if file.endswith(".wav"):
            file_path = os.path.join(root, file)
            emotion_code = file.split("-")[2]
            emotion = emotion_map.get(emotion_code)
            if emotion:
                try:
                    mfcc, mel, chroma = extract_features(file_path)
                    
                    mfcc_features.append(mfcc)
                    mel_features.append(mel)
                    chroma_features.append(chroma)
                    emotion_labels.append(emotion)
                except Exception as e:
                    print("Error in:", file_path, e)

def pad_feature(extracted_features):
    max_length = max(len(sub_feature) for sub_feature in extracted_features)

    padded_extracted_features = []
    for sub_feature in extracted_features:
        padding_needed = max_length - len(sub_feature)

        sub_feature_np = np.array(sub_feature)

        if padding_needed != 0:
            padded_sub_feature = np.pad(sub_feature_np, (0, padding_needed), 'constant', constant_values=0)
        else:
            padded_sub_feature = sub_feature_np
        
        padded_extracted_features.append(padded_sub_feature)

    return padded_extracted_features

padded_mfcc_features = pad_feature(mfcc_features)
padded_mel_features = pad_feature(mel_features)
padded_chroma_features = pad_feature(chroma_features)

mfcc_array = np.hstack(padded_mfcc_features)
mel_array = np.hstack(padded_mel_features)
chroma_array = np.hstack(padded_chroma_features)

os.makedirs("../results/features_and_labels", exist_ok=True)

np.save("../results/features_and_labels/X_features.npy", X)
np.save("../results/features_and_labels/y_labels.npy", y)

print("Saved X_features.npy and y_labels.npy to ../results/features_and_labels/")

### Step 02 : Handling Missing Values

In [ ]:
X = np.load("../results/features_and_labels/X_features.npy", allow_pickle=False)
y = np.load("../results/features_and_labels/y_labels.npy", allow_pickle=True)

df = pd.DataFrame(X)
df["Emotion"] = y

missing_counts = df.drop(columns="Emotion").isnull().sum()

features = df.drop(columns=["Emotion"])
df[features.columns].fillna(df[features.columns].mean(), inplace=True)

X = df.drop(columns="Emotion")
y = df["Emotion"]

np.save("../results/features_and_labels/X_features.npy", X)
np.save("../results/features_and_labels/y_labels.npy", y)

print("Saved X_features.npy and y_labels.npy to ../results/features_and_labels/")

### Step 03 : Lable Encoding the Target Variable

In [ ]:
X = np.load("../results/features_and_labels/X_features.npy", allow_pickle=False)
y = np.load("../results/features_and_labels/y_labels.npy", allow_pickle=True)

df = pd.DataFrame(X)
df["Emotion"] = y

unique_emotions, counts = np.unique(y, return_counts=True)

label_encoder = LabelEncoder()
y_label_encoded = label_encoder.fit_transform(y)

y_mapped = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

X = df.drop(columns="Emotion")
y = y_label_encoded

np.save("../results/features_and_labels/X_features.npy", X)
np.save("../results/features_and_labels/y_labels.npy", y)

print("Saved X_features.npy and y_labels.npy to ../results/features_and_labels/")

### Step 04 : Removing Outliers

In [ ]:
X = np.load("../results/features_and_labels/X_features.npy", allow_pickle=False)
y = np.load("../results/features_and_labels/y_labels.npy", allow_pickle=True)

df = pd.DataFrame(X)
df["Emotion"] = y

z_scores = np.abs(stats.zscore(df.drop(columns=["Emotion"])))

outlier_rows_z_score_method = (z_scores > 3).any(axis=1)

df_clean_z_score_method = df[~outlier_rows_z_score_method]

X = df_clean_z_score_method.drop(columns="Emotion")
y = df["Emotion"]

np.save("../results/features_and_labels/X_features.npy", X)
np.save("../results/features_and_labels/y_labels.npy", y)

print("Saved X_features.npy and y_labels.npy to ../results/features_and_labels/")

### Step 05 : Scaling Features

In [ ]:
X = np.load("../results/features_and_labels/X_features.npy", allow_pickle=False)
y = np.load("../results/features_and_labels/y_labels.npy", allow_pickle=True)

df = pd.DataFrame(X)
df["Emotion"] = y

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X = df.drop(columns="Emotion")
y = df["Emotion"]

np.save("../results/features_and_labels/X_features.npy", X)
np.save("../results/features_and_labels/y_labels.npy", y)

print("Saved X_features.npy and y_labels.npy to ../results/features_and_labels/")

### Step 06 : Balancing the Dataset by Oversampling

In [ ]:
X = np.load("../results/features_and_labels/X_features.npy", allow_pickle=False)
y = np.load("../results/features_and_labels/y_labels.npy", allow_pickle=True)

counts = Counter(y)

df = pd.DataFrame(list(zip(list(X), y)), columns=["Features", "Emotion"])

max_size = df["Emotion"].value_counts().max()

dfs = []
for label, group in df.groupby("Emotion"):
    dfs.append(resample(group, replace=True, n_samples=max_size, random_state=42))
df_balanced = pd.concat(dfs)

df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

X = df_balanced.drop(columns="Emotion")
y = df_balanced["Emotion"]

np.save("../results/features_and_labels/X_features.npy", X)
np.save("../results/features_and_labels/y_labels.npy", y)

print("Saved X_features.npy and y_labels.npy to ../results/features_and_labels/")

### Step 07 : Adding important Data as new Features (Feature Engineering)

In [ ]:
X = np.load("../results/features_and_labels/X_features.npy", allow_pickle=False)
y = np.load("../results/features_and_labels/y_labels.npy", allow_pickle=True)

df = pd.DataFrame(X)
df["Emotion"] = y

df["Feature Mean"] = df.drop(columns=["Emotion"]).mean(axis=1)
df["Feature Std"] = df.drop(columns=["Emotion"]).std(axis=1)
df["Feature Skewness"] = df.drop(columns=["Emotion"]).skew(axis=1)
df["Feature Kurtosis"] = df.drop(columns=["Emotion"]).kurt(axis=1)

X = df.drop(columns="Emotion")
y = df["Emotion"]

np.save("../results/features_and_labels/X_features.npy", X)
np.save("../results/features_and_labels/y_labels.npy", y)

print("Saved X_features.npy and y_labels.npy to ../results/features_and_labels/")